In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bisect
import glob
import seaborn as sns
%matplotlib
# Using 2016 data - stops and breakpoints

# Using the first week of Feb - 2016 - '01 - 31 - 16 to 02 - 06 - 16'
final_df = pd.read_csv('./travel_times_2016/fahrzeiten_soll_ist_20160131_20160206.csv')


#########################
# Using the first quarter's data (beginning of Jan - end of March)
# list_of_dfs = []
# for i in range(12):
# #     df = pd.read_csv(str(i+1)+".csv")
#     # Use the following and comment the above for friday's dataset
#     df = pd.read_csv(str(i+1)+".csv")
#     list_of_dfs.append(df)

# final_df = pd.concat(list_of_dfs)
# final_df.drop(['Unnamed: 0'], axis=1, inplace=True)
#########################


# Bus stop
df_haltepunkt = pd.read_csv("./travel_times_2016/haltepunkt.csv")

# Bus Station
df_haltestelle = pd.read_csv("./travel_times_2016/haltestelle.csv")
####################
# Some useful atts #
####################

# Target attributes - from: in seconds
target_arrival_from = "soll_an_von"

# Actual attributes - from: in seconds
actual_arrival_from = "ist_an_von"

# nominal departure from in seconds
target_departure_from = "soll_ab_von"

# actual  departure from in seconds
actual_departure_from = "ist_ab_von"
#######################################

#######################################
# Target attributes - from: in seconds
target_arrival_to = "soll_an_nach"

# Actual attributes - from: in seconds
actual_arrival_to = "ist_an_nach1"

# nominal departure from in seconds
target_departure_to = "soll_ab_nach"

# actual  departure from in seconds
actual_departure_to = "ist_ab_nach"
#####################################

#####################################

# key to haltestelle from and to
halt_id_from = "halt_id_von"
halt_id_to = "halt_id_nach"

############################
# key to haltepunkt from and to
halt_punkt_id_from = "halt_punkt_id_von"
halt_punkt_id_to = "halt_punkt_id_nach"

###########################

final_df.head()

Using matplotlib backend: Qt5Agg


,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,...,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach
0,2,1,31.01.16,3014,3,1,478,0,BEZI,31.01.16,...,18072,11,1,11,BEZI - BTIE für Ausfahrt,114628,1306,1502,2829,3312
1,2,1,31.01.16,3014,3,2,2556,0,STAU,31.01.16,...,18072,11,1,11,BEZI - BTIE für Ausfahrt,114628,1502,2657,3312,6066
2,2,1,31.01.16,3014,3,6,615,0,BURK,31.01.16,...,18072,11,1,11,BEZI - BTIE für Ausfahrt,114628,2590,1565,6341,3536
3,2,1,31.01.16,3014,3,5,601,0,BOES,31.01.16,...,18072,11,1,11,BEZI - BTIE für Ausfahrt,114628,2228,2590,6211,6341
4,2,1,31.01.16,3014,3,4,1861,0,PARA,31.01.16,...,18072,11,1,11,BEZI - BTIE für Ausfahrt,114628,1528,2228,3390,6211


# Limiting the dataset:

In [2]:

# a = final_df[final_df['betriebsdatum'] == '02.02.16'] # Limiting to one particular day-uncomment to restrict
a = final_df.copy(deep=True) # if above is uncommented, comment this one

# a = a[a['betriebsdatum'] == '01.02.16']



In [3]:
# top 5 most traveled routes
a['fahrweg_id'].value_counts()[:5]

18305    31321
18306    30555
21424    30401
21423    29078
21385    27161
Name: fahrweg_id, dtype: int64

In [4]:
a = a[a['fahrweg_id'] == 18306]
len(set(a['seq_nach']))

33

In [5]:
# Sorting the values by target arrivals from, individual routes ID, and sequence of stops

b = a.sort_values(by = ['betriebsdatum', target_arrival_from, 'fahrt_id', 'seq_von'])

# Remove trips where the dates of arrival and departure is different
b = b[b['betriebsdatum'] == b['datum_nach']]

# Define travel time and stationary

In [6]:
target_stationary = 'target_stationary'
target_travel_time = 'target_travel_time'

b[target_stationary] = b[target_departure_from] - b[target_arrival_from]
b[target_travel_time] = b[target_arrival_to] - b[target_departure_from]

b[target_stationary] = b[target_departure_from] - b[target_arrival_from]
b['actual'] = b[actual_arrival_to] - b[actual_departure_from]

In [7]:
# # Limiting to a specific stop:
# c = b[b['halt_punkt_diva_nach'] == 2544]
c = b.copy(deep=True)
c

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,...,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,target_stationary,target_travel_time,actual
201684,11,2,01.02.16,2050,2,1,2549,0,REHA,01.02.16,...,2,REHA - ZAUZ,114204,2092,2312,4779,5284,0,54,52
201683,11,2,01.02.16,2050,2,2,894,1,ENZE,01.02.16,...,2,REHA - ZAUZ,114204,2312,2121,5284,4872,12,72,88
201682,11,2,01.02.16,2050,2,3,421,1,BALG,01.02.16,...,2,REHA - ZAUZ,114204,2121,2295,4872,5247,18,66,71
201681,11,2,01.02.16,2050,2,4,580,1,BURG,01.02.16,...,2,REHA - ZAUZ,114204,2295,2110,5247,4832,12,48,50
201680,11,2,01.02.16,2050,2,5,2879,1,WETL,01.02.16,...,2,REHA - ZAUZ,114204,2110,2308,4832,5273,12,42,45
201679,11,2,01.02.16,2050,2,6,1138,1,HEDW,01.02.16,...,2,REHA - ZAUZ,114204,2308,2113,5273,9261,12,60,64
201678,11,2,01.02.16,2050,2,7,1144,1,HEGI,01.02.16,...,2,REHA - ZAUZ,114204,2113,2275,9261,5186,12,48,50
201744,11,2,01.02.16,2050,2,8,2388,1,SIGN,01.02.16,...,2,REHA - ZAUZ,114204,2275,2084,5186,9279,12,66,91
201743,11,2,01.02.16,2050,2,9,1453,1,KREU,01.02.16,...,2,REHA - ZAUZ,114204,2084,2111,9279,4834,18,102,108
201742,11,2,01.02.16,2050,2,10,3039,1,BSTA,01.02.16,...,2,REHA - ZAUZ,114204,2111,1565,4834,3552,18,78,66


## TESTING _ BELOW

In [8]:
# sorting days well:
d = pd.to_datetime(c['betriebsdatum'], format='%d.%m.%y', dayfirst=True)
# Monday == 0 … Sunday == 6
c['day_of_week'] = [i.weekday() for i in d]
c['days_colors'] = c['day_of_week'].apply(lambda x: 'monday' if x == 0 else 'tuesday' if x == 1 else 'wednesday' 
                       if x == 2 else 'thursday' if x == 3 else 'friday' if x == 4 else 'saturday' if x == 5 else 'sunday' if x == 6 else 'unk')

In [9]:
c = c[c['day_of_week'] != 5]
c = c[c['day_of_week'] != 6]

In [10]:
def someFunc(x):
    x = np.array( [ num for num in x if num >= 0 ] )
    return np.bincount(x).argmax()

In [33]:
%matplotlib

# Aggfunc determines what to do with dups

time_period = 600
starting_time = 0
# indicates 30 mins intervals
# Number of bins:
n_bins = int(np.floor((86400 - starting_time)/time_period))
labels = [i for i in range(n_bins - 1)]
# Finding the correct bins:
list_of_bins = [starting_time + time_period * i for i in range(n_bins)]
c['intervals'] = pd.cut(c[target_arrival_from], bins=list_of_bins, retbins=False, labels=labels)


fig, axes = plt.subplots(2, 2)

ax1 = axes[0, 0]
ax2 = axes[0, 1]
ax3 = axes[1, 0]
ax4 = axes[1, 1]




plt.xlabel('Bus Stop Number - ID')
plt.ylabel('Target Departure')
ax1.set_title('HeatMap - target departure vs. sequential stops, value: target travel time - Non-normalized ')
f = pd.pivot_table(c, values=target_travel_time, columns='seq_nach', index='intervals', aggfunc=np.mean)

# for i in f.columns.values:
#     print("{}:{}".format(i, f[i].std()))

ax3.set_title('HeatMap - target departure vs. sequential stops, value: target travel time - normalized')
sns.heatmap(f,fmt="f", robust=True,cbar_kws={"shrink": 1}, ax=ax1)
# normalizing
for i in f:
    min_val = f[i].min()
#     print(min_val)
    f[i] = f[i].apply(lambda x: x/min_val)
    
    
sns.heatmap(f,fmt="f", robust=True,cbar_kws={"shrink": 1}, ax=ax3)


    

plt.xlabel('Bus Stop Number - ID')
plt.ylabel('Target Departure')
ax2.set_title('HeatMap - target departure vs. sequential stops, value: actual travel time - Non - normalized')
e = pd.pivot_table(c, values='actual', columns='seq_nach', index='intervals', aggfunc=np.mean)
ax4.set_title('HeatMap - target departure vs. sequential stops, value: actual travel time - normalized')
sns.heatmap(e,fmt="f", robust=True,cbar_kws={"shrink": 1}, ax=ax2)
# normalizing
for i in e:
    min_val = e[i].min()
#     print(min_val)
    e[i] = e[i].apply(lambda x: x/min_val)
    
    
sns.heatmap(e,fmt="f", robust=True,cbar_kws={"shrink": 1}, ax=ax4)


Using matplotlib backend: Qt5Agg


In [12]:
%matplotlib

# Aggfunc determines what to do with dups

time_period = 120
starting_time = 0
# indicates 30 mins intervals
# Number of bins:
n_bins = int(np.floor((86400 - starting_time)/time_period))
labels = [i for i in range(n_bins - 1)]
# Finding the correct bins:
list_of_bins = [starting_time + time_period * i for i in range(n_bins)]
c['intervals'] = pd.cut(c[target_arrival_from], bins=list_of_bins, retbins=False, labels=labels)


fig, axes = plt.subplots(2, 2)

ax1 = axes[0, 0]
ax2 = axes[0, 1]
ax3 = axes[1, 0]
ax4 = axes[1, 1]




plt.xlabel('Bus Stop Number - ID')
plt.ylabel('Target Departure')
ax1.set_title('HeatMap - target departure vs. sequential stops, value: target travel time - Non-normalized ')
f = pd.pivot_table(c, values=target_travel_time, columns='seq_nach', index='intervals', aggfunc=np.max)

ax3.set_title('HeatMap - target departure vs. sequential stops, value: target travel time - normalized')
sns.heatmap(f,fmt="f", robust=True,cbar_kws={"shrink": 1}, ax=ax1)
# normalizing
for i in f:
    min_val = f[i].min()
#     print(min_val)
    f[i] = f[i].apply(lambda x: x/min_val)
    
    
sns.heatmap(f,fmt="f", robust=True,cbar_kws={"shrink": 1}, ax=ax3)


    

plt.xlabel('Bus Stop Number - ID')
plt.ylabel('Target Departure')
ax2.set_title('HeatMap - target departure vs. sequential stops, value: actual travel time - Non - normalized')
e = pd.pivot_table(c, values='actual', columns='seq_nach', index='intervals', aggfunc=np.max)
ax4.set_title('HeatMap - target departure vs. sequential stops, value: actual travel time - normalized')
sns.heatmap(e,fmt="f", robust=True,cbar_kws={"shrink": 1}, ax=ax2)
# normalizing
for i in e:
    min_val = e[i].min()
#     print(min_val)
    e[i] = e[i].apply(lambda x: x/min_val)
    
    
sns.heatmap(e,fmt="f", robust=True,cbar_kws={"shrink": 1}, ax=ax4)


Using matplotlib backend: Qt5Agg


In [13]:
%matplotlib

# 3d plot - X: target dep time, Y: sequential stops - ID in that route, and hight is the target travel time 
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(c[target_departure_from], c[halt_punkt_id_from], c[target_travel_time])

Using matplotlib backend: Qt5Agg


In [14]:
# sorting days well:
d = pd.to_datetime(c['betriebsdatum'], format='%d.%m.%y', dayfirst=True)
# Monday == 0 … Sunday == 6
c['day_of_week'] = [i.weekday() for i in d]
c['days_colors'] = c['day_of_week'].apply(lambda x: 'monday' if x == 0 else 'tuesday' if x == 1 else 'wednesday' 
                       if x == 2 else 'thursday' if x == 3 else 'friday' if x == 4 else 'saturday' if x == 5 else 'sunday' if x == 6 else 'unk')

In [15]:

%matplotlib

# Plotting target dept in X - weekdays on y


plt.scatter(c['days_colors'], c[target_departure_from], c=c['day_of_week'])

plt.legend(['0 : Monday, 1 : Tuesday, ..., 6 : Sunday'])
# c['index'] = range(len(c))
# pd.options.display.max_rows = None
# display(c)

Using matplotlib backend: Qt5Agg


In [16]:
print(len(c))
print(len(set(c[halt_punkt_id_from])))

22429
33


In [17]:
# Plotting target arrivals vs. target depearture, color days of week

%matplotlib
plt.title('target arrival from  vs target travel time')
plt.scatter(c[target_arrival_from], c[target_travel_time], c=c['day_of_week'])

Using matplotlib backend: Qt5Agg


In [18]:
# Plotting instance vs. target depearture, color days of week

%matplotlib
plt.title('index from  vs target departure from')
plt.scatter(range(len(c)), c[target_travel_time], c=c['day_of_week'])

Using matplotlib backend: Qt5Agg


In [19]:



time_period = 1800
starting_time = 0
# indicates 30 mins intervals
# Number of bins:
n_bins = int(np.floor((86400 - starting_time)/time_period))
labels = [i for i in range(n_bins - 1)]
# Finding the correct bins:
list_of_bins = [starting_time + time_period * i for i in range(n_bins)]
# intervals = pd.interval_range(start=21600, end=86400, periods=n_bins)
b['intervals'] = pd.cut(b[target_arrival_from], bins=list_of_bins, retbins=False, labels=labels)
# plt.scatter(df_2['intervals'], df_2['delay'])
# plt.xlim(0, n_bins+2)
# plt.show()
# to make the plotting more readible
labels_text = [
'0:00','0:30','1:00','1:30','2:00','2:30','3:00','3:30','4:00','4:30','5:00','5:30','6:00','6:30','7:00','7:30','8:00','8:30','9:00',
'9:30','10:00','10:30','11:00','11:30','12:00','12:30','13:00','13:30','14:00','14:30','15:00',
'15:30','16:00','16:30','17:00','17:30','18:00','18:30','19:00','19:30','20:00','20:30','21:00',
'21:30','22:00','22:30','23:00','23:30']


avgs_stationary = []
# Averaging:
for i in labels:
    avgs_stationary.append(b[b['intervals'] == i].mean()[target_stationary])
for i in range(len(avgs_stationary)):
    if np.isnan(avgs_stationary[i]):
        avgs_stationary[i] = 0

avgs_travel_time = []
# Averaging:
for i in labels:
    avgs_travel_time.append(b[b['intervals'] == i].mean()[target_travel_time])
for i in range(len(avgs_travel_time)):
    if np.isnan(avgs_travel_time[i]):
        avgs_travel_time[i] = 0
        
# plt.scatter(labels, avgs)
# plt.show()

# plotting stationary
f_1 = plt.figure(1)

plt.xlabel('time of the day')
plt.ylabel('stationary time in seconds - target')
plt.title('Target stationary times in seconds')
plt.xticks(labels, labels_text)
plt.bar(labels, avgs_stationary)
plt.show()

# plotting stationary
f_2 = plt.figure(2)

plt.xlabel('time of the day')
plt.ylabel('travel time in seconds - target')
plt.title('Target travel times in seconds')
plt.xticks(labels, labels_text)
plt.bar(labels, avgs_travel_time)
plt.show()

In [32]:
# heatmap viz
pd.pivot_table(data='')

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,...,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach
38813,11,2,31.01.16,3019,11,22,1522,1,LAUI,31.01.16,...,18306,2,1,2,REHA - ZAUZ,110878,2085,1309,4752,2851
38814,11,2,31.01.16,3019,11,21,2158,3,SPLA,31.01.16,...,18306,2,1,2,REHA - ZAUZ,110878,1539,2085,3432,4752
38815,11,2,31.01.16,3019,11,20,1468,1,KRON,31.01.16,...,18306,2,1,2,REHA - ZAUZ,110878,2087,1539,4756,3432
38816,11,2,31.01.16,3019,11,19,435,1,BECK,31.01.16,...,18306,2,1,2,REHA - ZAUZ,110878,2086,2087,4755,4756
38817,11,2,31.01.16,3019,11,18,2474,1,STAM,31.01.16,...,18306,2,1,2,REHA - ZAUZ,110878,2088,2086,4761,4755
38818,11,2,31.01.16,3019,11,17,413,1,BQUA,31.01.16,...,18306,2,1,2,REHA - ZAUZ,110878,1542,2088,3454,4761
38819,11,2,31.01.16,3019,11,16,416,1,BSTR,31.01.16,...,18306,2,1,2,REHA - ZAUZ,110878,1895,1542,4260,3454
38820,11,2,31.01.16,3019,11,15,2054,1,RAUG,31.01.16,...,18306,2,1,2,REHA - ZAUZ,110878,2035,1895,4602,4260
38821,11,2,31.01.16,3019,11,14,1861,5,PARA,31.01.16,...,18306,2,1,2,REHA - ZAUZ,110878,1528,2035,3386,4602
38822,11,2,31.01.16,3019,11,13,601,1,BOES,31.01.16,...,18306,2,1,2,REHA - ZAUZ,110878,2228,1528,6210,3386
